# Radon Multi-level Model

In this notebook, we use the radon data and the multi-level model for modeling it. 
Next, we train a decision-maker (neural network) to show that it is capable of correcting errors due to posterior approximation.
The model is implemented using the publicly available [Stan code](http://mc-stan.org/users/documentation/case-studies/radon.html), using the Minnesota subset of the data and also otherwise conforming to their details. 
The inference is carried out using automatic differentiation variational inference, and we split the data randomly into equally sized training and test set.

### Proposed Experiment

Since variational approximations are trained with iterative algorithms, we can construct a controlled experiment by terminating the inference algorithm early and varying the termination point. In general, approximations trained for shorter time are further away from the final approximation and the true posterior. 

### Imports

In [1]:
import time

In [2]:
import pystan
import pickle
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.nn as nn

In [4]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    env = torch.cuda
    device = torch.device('cuda')
    print("Using GPU")
else:
    torch.set_default_tensor_type('torch.FloatTensor')
    env = torch
    device = torch.device('cpu')
    print("Using CPU")
    
    
def totorch(array):
    return torch.tensor(array, dtype=env.float32)

Using CPU


In [5]:
from aux import tonumpy, parse_script_args, dict2str, print2
import losses
import regression_data
import radon

### Configuration

In [6]:
args = parse_script_args() # arguments can be passed in the format of NAME1=FLOATVAL1,NAME2=[STRVAL2],NAME3=INTVAL3,...

parsing: <-f>


In [7]:
# optimization general parmeters
SEED = args.get("SEED", 5)
VI_NITER = int(args.get("VI_NITER", 1e6))
HMC_NITER = int(args.get("HMC_NITER", 1e6))

In [8]:
# regularization
REGULARIZATION = args.get("REGULARIZATION", "boot1").lower() # const/boot1
LAMBDA = args.get("LAMBDA", 1.)

In [9]:
# selected loss: tilted/squared/exptilted/expsquared
LOSS = args.get("LOSS", "tilted")
TILTED_Q = args.get("TILTED_Q", 0.5) # relevant only for tilted and exptilted

In [10]:
print("CONFIGURATION SUMMARY: %s" % dict2str(globals()))

CONFIGURATION SUMMARY: HMC_NITER=0 LAMBDA=1.0 LOSS=tilted REGULARIZATION=boot1 SEED=5 TILTED_Q=0.5 VI_NITER=10000


In [11]:
np.random.seed(SEED)
torch.manual_seed(SEED)

### Losses

In [12]:
loss, optimal_h_bayes_estimator = losses.LossFactory(**globals()).create(LOSS)
print2("> <%s> loss: %s with (analytical/Bayes estimator) h: %s" % 
        (LOSS, loss.__name__, optimal_h_bayes_estimator.__name__))

[LossFactory] Configuration: TILTED_Q=0.5 LINEX_C=None
> <tilted> loss: tilted_loss_fixedq with (analytical/Bayes estimator) h: tilted_optimal_h_fixedq


In [13]:
empirical_risk = lambda preds, y: loss(preds, totorch(y)).mean()

### Data

In [14]:
log_radon, n_county, county, u, floor_measure = regression_data.load_radon()

In [15]:
TRAIN_MASK = np.array([i%2==0 for i in range(len(log_radon))], dtype=bool)
TEST_MASK = ~TRAIN_MASK

COUNTY_TRAIN = county[TRAIN_MASK]
U_TRAIN = u[TRAIN_MASK]
FLOOR_MEASURE_TRAIN = floor_measure[TRAIN_MASK]
LOG_RADON_TRAIN = log_radon[TRAIN_MASK]

COUNTY_TEST = county[TEST_MASK]
U_TEST = u[TEST_MASK]
FLOOR_MEASURE_TEST = floor_measure[TEST_MASK]
LOG_RADON_TEST = log_radon[TEST_MASK]

In [16]:
Y_TRAIN, Y_TEST = LOG_RADON_TRAIN, LOG_RADON_TEST
print(Y_TRAIN.shape, Y_TEST.shape)

(460,) (459,)


### Inference

In [17]:
path = "radon_model.pkl"
try:
    print("Loading model from %s" % path)
    sm = pickle.load(open(path, "rb"))
except:
    print("Failed. Recomputing and saving model to %s" % path)
    sm = pystan.StanModel(model_code=radon.hierarchical_intercept)
    pickle.dump(sm, open(path, "wb"))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_09c24493b150e61b5f7babd2ed515f09 NOW.


Loading model from radon_model.pkl
Failed. Recomputing and saving model to radon_model.pkl


In [18]:
hierarchical_intercept_data = radon.create_data(LOG_RADON_TRAIN, n_county, 
                                                COUNTY_TRAIN, U_TRAIN, FLOOR_MEASURE_TRAIN)

### HMC evaluation

In [20]:
if HMC_NITER<=0:
    hmc_train_risk = 0.
    hmc_test_risk = 0.
else:
    hmc_fit = sm.sampling(data=hierarchical_intercept_data, iter=HMC_NITER, n_jobs=1)
    ys_train_hmc = totorch(radon.sample_predictive_y0_hmc(hmc_fit, FLOOR_MEASURE_TRAIN, U_TRAIN, COUNTY_TRAIN))
    ys_test_hmc = totorch(radon.sample_predictive_y0_hmc(hmc_fit, FLOOR_MEASURE_TEST, U_TEST, COUNTY_TEST))
    hstar_train_hmc = optimal_h_bayes_estimator(ys_train_hmc)
    hstar_test_hmc = optimal_h_bayes_estimator(ys_test_hmc)
    hmc_train_risk = empirical_risk(hstar_train_hmc, Y_TRAIN).item()
    hmc_test_risk = empirical_risk(hstar_test_hmc, Y_TEST).item()

In [21]:
print("HMC optimal risk: train:%.4f test:%.4f" % (hmc_train_risk, hmc_test_risk))

HMC optimal risk: train:0.0000 test:0.0000


### VI Evaluation

In [22]:
#vi_fit = sm.vb(data=hierarchical_intercept_data, iter=1000000, tol_rel_obj=0.0001)
vi_fit = sm.vb(data=hierarchical_intercept_data, iter=VI_NITER, tol_rel_obj=0.0001)

In [23]:
ys_train = totorch(radon.sample_predictive_y0_vi(vi_fit, FLOOR_MEASURE_TRAIN, U_TRAIN, COUNTY_TRAIN))
ys_test = totorch(radon.sample_predictive_y0_vi(vi_fit, FLOOR_MEASURE_TEST, U_TEST, COUNTY_TEST))

In [24]:
hstar_train = optimal_h_bayes_estimator(ys_train)
hstar_test = optimal_h_bayes_estimator(ys_test)

In [25]:
vi_train_risk = empirical_risk(hstar_train, Y_TRAIN).item()
vi_test_risk = empirical_risk(hstar_test, Y_TEST).item()
print("VI optimal risk: train:%.4f test:%.4f" % (vi_train_risk, vi_test_risk))

VI optimal risk: train:0.2686 test:0.2884


### Regularization

In [26]:
import regression_regularization as regularization

HSTAR_TRAIN = hstar_train # default regularization mean

print("[regularization] Using regularization: %s with lambda=%s" % (REGULARIZATION, LAMBDA))
if REGULARIZATION.startswith("boot"):     
    print("[regularization] Bootstrap-based regularization")    
    stan_data_generator = radon.yield_data_bootstrap(LOG_RADON_TRAIN, n_county, COUNTY_TRAIN, 
                                                     U_TRAIN, FLOOR_MEASURE_TRAIN)
    sample_predictive_y0_train = lambda vi_fit: radon.sample_predictive_y0_vi(vi_fit, 
                                                    FLOOR_MEASURE_TRAIN, U_TRAIN, COUNTY_TRAIN)
    sample_predictive_y0_test = lambda vi_fit: radon.sample_predictive_y0_vi(vi_fit, 
                                                    FLOOR_MEASURE_TEST, U_TEST, COUNTY_TEST)
    optimal_h_bayes_estimator_np = lambda ys: tonumpy(optimal_h_bayes_estimator(totorch(ys)))

    hstar1_train, _ = regularization.get_bootstrap_decisions(sm, stan_data_generator, 
                                        sample_predictive_y0_train, sample_predictive_y0_test, 
                                        optimal_h_bayes_estimator_np, num_repetitions=5, iter=VI_NITER)        
    LAMBDA_TRAIN = LAMBDA * 1.0/(2. * hstar1_train.std(0)**2)
        
    if REGULARIZATION.startswith("boot1"):
        print("[regularization] Bootstrap-based regularization: overwritting regularization mean")
        HSTAR_TRAIN = totorch( hstar1_train.mean(0) )
        
elif REGULARIZATION.startswith("const"): 
    LAMBDA_TRAIN = regularization.get_regularization_constant(ys_train, LAMBDA)
elif REGULARIZATION.startswith("std"): 
    LAMBDA_TRAIN = regularization.get_regularization_std(ys_train, LAMBDA)
elif REGULARIZATION.startswith("qdiff"): 
    LAMBDA_TRAIN = regularization.get_regularization_qdiff(ys_train, LAMBDA)
else:    
    raise Exception("[regularization] Unknown regularization method name: %s" % method)    

LAMBDA_TRAIN = totorch(LAMBDA_TRAIN)

[regularization] Using regularization: boot1 with lambda=1.0
[regularization] Bootstrap-based regularization
[get_bootstrap_decisions] 0/5


[get_bootstrap_decisions] 1/5


[get_bootstrap_decisions] 2/5


[get_bootstrap_decisions] 3/5


[get_bootstrap_decisions] 4/5


[regularization] Bootstrap-based regularization: overwritting regularization mean


### Quantile optimization

In [27]:
def training_empirical_risk(q):    
    q = max(0., min(1., q[0]))
    h = losses.tilted_optimal_h(ys_train, q) # obtaing a quantile in a very convoluted way :)
    risk = empirical_risk(h, Y_TRAIN).item() 
    regularizer = ( LAMBDA_TRAIN * (h-HSTAR_TRAIN)**2 ).mean()
    obj = risk + regularizer
    print("evaluating training risk @ q=%.2f => risk=%.3f => obj=%.3f" % (q, risk, obj))
    return obj                      

In [28]:
from scipy.optimize import minimize
res = minimize(training_empirical_risk, [TILTED_Q], 
               method='Nelder-Mead', options={'xtol': 1e-5, 'disp': True, "maxiter": 100})

evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.53 => risk=0.268 => obj=0.648
evaluating training risk @ q=0.47 => risk=0.271 => obj=0.733
evaluating training risk @ q=0.51 => risk=0.268 => obj=0.597
evaluating training risk @ q=0.49 => risk=0.270 => obj=0.645
evaluating training risk @ q=0.51 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.49 => risk=0.269 => obj=0.594
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.594
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.594
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.579
evaluating training risk @ q=0.50 => risk=0.269 => obj=0.594
evaluating training risk

In [29]:
q = max(0., min(1., res["x"][0]))
q_train_risk = empirical_risk(losses.tilted_optimal_h(ys_train, q), Y_TRAIN).item() 
q_test_risk = empirical_risk(losses.tilted_optimal_h(ys_test, q), Y_TEST).item() 
print("Quantile optimal risk: train:%.3f test:%.4f" % (q_train_risk, q_test_risk))

Quantile optimal risk: train:0.269 test:0.2884


### Decision maker

In [30]:
torch.manual_seed(SEED)

In [31]:
# Prepare features 
Quantiles = np.array([np.percentile(ys_train, int(q*100), axis=0) for q in np.arange(0., 1.01, 0.05)])
X_train = Quantiles
Quantiles = np.array([np.percentile(ys_test, int(q*100), axis=0) for q in np.arange(0., 1.01, 0.05)])
X_test = Quantiles

X_train, X_test = totorch(X_train.T), totorch(X_test.T)

In [32]:
decision_maker = nn.Sequential(
  nn.Linear(X_train.shape[1], 20),
  nn.ReLU(),
  nn.Linear(20, 10),
  nn.ReLU(),
  nn.Linear(10, 1)
)

print(decision_maker)

Sequential(
  (0): Linear(in_features=21, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
  (3): ReLU()
  (4): Linear(in_features=10, out_features=1, bias=True)
)


In [33]:
optimizer = torch.optim.Adam(decision_maker.parameters(), lr=0.01)
start = time.time()
for i in range(20000):
        h = decision_maker(X_train).view(-1)
        train_loss = empirical_risk(h, Y_TRAIN)  
        regularizer = ( totorch(LAMBDA_TRAIN) * (h-HSTAR_TRAIN)**2 ).mean()

        optimizer.zero_grad()
        (train_loss+regularizer).backward()
        optimizer.step()

        if i%1000==0:
            print("[%.2fs] %i. iteration: training batch risk: %.3f regularizer: %.3f train risk: %.3f test risk: %.3f" % 
              (time.time()-start, i, train_loss.item(), regularizer.item(),
               empirical_risk(decision_maker(X_train).view(-1), Y_TRAIN).item(),
               empirical_risk(decision_maker(X_test).view(-1), Y_TEST).item()))

[0.02s] 0. iteration: training batch risk: 0.754 regularizer: 113.243 train risk: 0.743 test risk: 0.712


/home/anonymized/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


[3.40s] 1000. iteration: training batch risk: 0.267 regularizer: 0.233 train risk: 0.267 test risk: 0.289
[6.13s] 2000. iteration: training batch risk: 0.267 regularizer: 0.223 train risk: 0.267 test risk: 0.289
[8.71s] 3000. iteration: training batch risk: 0.267 regularizer: 0.218 train risk: 0.267 test risk: 0.289
[11.15s] 4000. iteration: training batch risk: 0.266 regularizer: 0.211 train risk: 0.266 test risk: 0.289
[13.76s] 5000. iteration: training batch risk: 0.266 regularizer: 0.196 train risk: 0.266 test risk: 0.289
[15.88s] 6000. iteration: training batch risk: 0.266 regularizer: 0.194 train risk: 0.266 test risk: 0.290
[18.58s] 7000. iteration: training batch risk: 0.266 regularizer: 0.192 train risk: 0.266 test risk: 0.289
[20.62s] 8000. iteration: training batch risk: 0.266 regularizer: 0.192 train risk: 0.266 test risk: 0.290
[23.89s] 9000. iteration: training batch risk: 0.266 regularizer: 0.192 train risk: 0.266 test risk: 0.290
[27.86s] 10000. iteration: training batc

In [34]:
dm_train_risk = empirical_risk(decision_maker(X_train).view(-1), Y_TRAIN).item()
dm_test_risk = empirical_risk(decision_maker(X_test).view(-1), Y_TEST).item()
print("DM optimal risk: train:%.3f test:%.4f" % (dm_train_risk, dm_test_risk))

DM optimal risk: train:0.266 test:0.2904


### Store evaluation results

In [35]:
results = [[SEED, LOSS, TILTED_Q, REGULARIZATION, LAMBDA, VI_NITER, HMC_NITER,
            vi_train_risk, vi_test_risk, 
            q_train_risk, q_test_risk, 
            dm_train_risk, dm_test_risk,
            hmc_train_risk, hmc_test_risk]]

In [36]:
path = "radon_%i_%s_%s_%s_%s_%s.csv" % \
            (SEED, LOSS, TILTED_Q, REGULARIZATION, LAMBDA, VI_NITER)
print("Saving to %s" % path)
pd.DataFrame(results).to_csv(path, header=False, index=False)

Saving to radon_5_tilted_0.5_boot1_1.0_10000.csv
